In [1]:
#import
import math as math
import numpy as np
import collections
from collections import Counter
import json
from operator import itemgetter
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed


Using TensorFlow backend.


In [9]:
# setup and load input text
SEQ_LENGTH = 50
BATCH_SIZE = 30
LAYER_NUM = 2
HIDDEN_DIM = 500
GENERATE_LENGTH = 500
NB_EPOCH = 140 # standard:20
GENERATE_FREQ=10 #generate text every X epoch
MODE = "train"
WEIGHTS = ""

data_dir = 'resources/LOTR.txt'
text = open(data_dir, "r").read()

In [3]:
#convert all data to lowecase and skip unknown chars
data = []

for i in range(len(text)):
    t = text[i]
    if t.isalpha() or \
                    t == "." or \
                    t == "," or \
                    t == "!" or \
                    t == "?" or \
                    t == ":" or \
                    t == "-" or \
                    t == " " or \
                    t == "'":
        data.append(t)

ch = Counter(data)
ch = collections.OrderedDict(sorted(ch.items(), key=itemgetter(1), reverse=True))

#output dictionaries
gen_ix_to_char = {ix: char for ix, char in enumerate(ch)}
char_to_ix = {char: ix for ix, char in enumerate(ch)}

text_ix = [char_to_ix[value] for value in data] 

print("number of input text", len(text))
print("number of deleted chars", len(text) - len(data))
print("number of chars", len(data))
print("length of dictionary", len(gen_ix_to_char))


number of input text 2470966
number of deleted chars 4360
number of chars 2466606
length of dictionary 60


In [4]:
#prepare zero matrixes for each sequence
VOCAB_SIZE = int(len(gen_ix_to_char))
print('Data length: {} chars'.format(len(text_ix)))
print('Vocabulary size: {} chars'.format(VOCAB_SIZE))
num_of_seq=int(math.ceil(len(text_ix) / SEQ_LENGTH))
X = np.zeros((num_of_seq, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((num_of_seq, SEQ_LENGTH, VOCAB_SIZE))

print("for[0:" + str(math.floor(len(text_ix) / SEQ_LENGTH)) + "]")



Data length: 2466606 chars
Vocabulary size: 60 chars
for[0:49332]


In [5]:
#function for text generation
range_num=int(math.floor(len(text_ix) / SEQ_LENGTH))
for i in range(range_num):
        if i % 5000 == 0: print(i, '/', range_num)
        X_sequence_ix = text_ix[i * SEQ_LENGTH:(i + 1) * SEQ_LENGTH]
        input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))

        for j in range(SEQ_LENGTH):
            input_sequence[j][X_sequence_ix[j]] = 1.
            X[i] = input_sequence

        y_sequence_ix = text_ix[i * SEQ_LENGTH + 1:(i + 1) * SEQ_LENGTH + 1]
        target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
        for j in range(SEQ_LENGTH):
            target_sequence[j][y_sequence_ix[j]] = 1.
            y[i] = target_sequence

0 / 49332
5000 / 49332
10000 / 49332
15000 / 49332
20000 / 49332
25000 / 49332
30000 / 49332
35000 / 49332
40000 / 49332
45000 / 49332


In [6]:
#Creating and compiling the Network

model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [7]:
#function for text generation
def generate_text(model, length, vocab_size, gen_ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    # print(ix)
    y_char = [gen_ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(gen_ix_to_char[ix[-1]], sep='', end='', flush=True)

        ix = np.argmax(model.predict(X[:, :i + 1, :])[0], 1)
        y_char.append(gen_ix_to_char[ix[-1]])
    return ('').join(y_char)

In [10]:
#Generate some sample before training to know how bad it is!
model.save('checkpoint_layer_{}_hidden_{}_test.hdf5'.format(LAYER_NUM, HIDDEN_DIM))
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, gen_ix_to_char)

NBATFaFadTDfT--T:p:::f:::OOOOOOOOy!!!!!IIIIj--gwRwwqqqgEEEEgEyNkCCppppEEE!!CGGNBBABNTATFFppppEEE!!!GGNNBBAATFTTpppwwwqqqqEEgEEEk!CCkssssksYYYYYtttttIIIIIwwwYYqtqqggEEEENk'kypppEEE!CCCGGNBBBABTTT,,vvwwwwqqqqqgEEEEEk'ypppEEECCCCpGpGBBBBZZNBBZZNNBKKKKKZZNYYYtttttKKKKKKrYYYtttttIIIIwwYYttttIIppwIwIwRqqqEEgEENkkksssskssksssksYYYYYtttttIIIIIwwwYYqtqqggEEEENk'kypppEEE!CCCGGNBBBABTTT,,vvwwwwqqqqqgEEEEEk'ypppEEECCCCpGpGBBBBZZNBBZZNNBKKKKKZZNYYYtttttKKKKKKrYYYtttttIIIIwwYYttttIIppwIwIwRqqqEEgEENkkksssskss

"NBATFaFadTDfT--T:p:::f:::OOOOOOOOy!!!!!IIIIj--gwRwwqqqgEEEEgEyNkCCppppEEE!!CGGNBBABNTATFFppppEEE!!!GGNNBBAATFTTpppwwwqqqqEEgEEEk!CCkssssksYYYYYtttttIIIIIwwwYYqtqqggEEEENk'kypppEEE!CCCGGNBBBABTTT,,vvwwwwqqqqqgEEEEEk'ypppEEECCCCpGpGBBBBZZNBBZZNNBKKKKKZZNYYYtttttKKKKKKrYYYtttttIIIIwwYYttttIIppwIwIwRqqqEEgEENkkksssskssksssksYYYYYtttttIIIIIwwwYYqtqqggEEEENk'kypppEEE!CCCGGNBBBABTTT,,vvwwwwqqqqqgEEEEEk'ypppEEECCCCpGpGBBBBZZNBBZZNNBKKKKKZZNYYYtttttKKKKKKrYYYtttttIIIIwwYYttttIIppwIwIwRqqqEEgEENkkksssskssk"

In [ ]:
#Training if there is no trained weights specified
nb_epoch = 0
if MODE == 'train' or WEIGHTS == '':
    while nb_epoch<NB_EPOCH:
        print('\n\nEpoch: {}\n'.format(nb_epoch))
        model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=1)
        nb_epoch += 1


        # generate every GENERATE_FREQth epoch
        if nb_epoch % GENERATE_FREQ == 0 or nb_epoch==1 or nb_epoch==5 or nb_epoch==10:
            gen_text = generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, gen_ix_to_char)
            gen_text = str(gen_text)
           
            with open('char_{}_hidden_{}_epoch_{}.json'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch), 'w') as fp:
                json.dump(gen_text, fp)

            model.save_weights('char_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))
            # model.save('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))





Epoch: 0

Epoch 1/1
 9150/49333 [====>.........................] - ETA: 8:58 - loss: 2.5585